In [ ]:
from importlib import reload
from msclap.dataset import get_data_loader
from tqdm import tqdm
import pandas as pd
import torch
from msclap import CLAP
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
train_csv_path = '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/csv/train.csv'
val_csv_path = '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/csv/val.csv'

# origin_train = pd.read_csv(train_csv_path)
# origin_val = pd.read_csv(val_csv_path)
# origin_train

train_loader = get_data_loader(csv_path=train_csv_path, batch_size=8, mode='train')
val_loader = get_data_loader(csv_path=val_csv_path, batch_size=8, mode='val')


In [6]:
for audio_files, class_labels in train_loader:
    print(audio_files)
    print(class_labels)
    break

# from IPython.display import display, Audio
# audio_files = [
#     '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill/0000097_Hit.mp3',
#     '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill/0000097_Use.mp3',
#     '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill/0000099_Hit.mp3',
#     '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill/0000099_Use.mp3',
#     '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill/0000100_Hit.mp3',
# ]
# display(Audio(audio_files[2]))

# class_labels = [
#     'This Sound is class of sharp blade',
#     'This Sound is class of blunt blade',
#     'fire',
#     'blunt weapon',
#     'ax',
# ]

['/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/65121052_Use.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/135001000_Hit.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/400001064_Hit.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/63141012_Hit.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/31211003_Use.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/80003538_Use.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/35141002_Hit.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/22110023_Use.mp3']
['The sharp, forceful sound of a magical weapon, like a wand or staff, conjuring the 

In [ ]:

# Load model (Choose between versions '2022' or '2023')
# The model weight will be downloaded automatically if `model_fp` is not specified
clap_model = CLAP(version = '2023', use_cuda=True)

# # Extract text embeddings
# text_embeddings = clap_model.get_text_embeddings(class_labels = class_labels)

# # Extract audio embeddings
# audio_embeddings = clap_model.get_audio_embeddings(audio_files=audio_files[:1])

# # Compute similarity between audio and text embeddings
# similarities = clap_model.compute_similarity(audio_embeddings, text_embeddings)
# print(f"class_labels : {class_labels}")
# similarities

In [ ]:
audio_encoder_finetune = False
caption_encoder_finetune = False
projection_layer_finetune = True

if audio_encoder_finetune:
    clap_model.clap.audio_encoder.eval()
    for n, p in clap_model.clap.audio_encoder.named_parameters():
        p.requires_grad = True
else:
    clap_model.clap.audio_encoder.train()
    for n, p in clap_model.clap.audio_encoder.named_parameters():
        p.requires_grad = False
    

if caption_encoder_finetune:
    clap_model.clap.caption_encoder.eval()
    for n, p in clap_model.clap.caption_encoder.named_parameters():
        p.requires_grad = True
else:
    clap_model.clap.caption_encoder.train()
    for n, p in clap_model.clap.caption_encoder.named_parameters():
        p.requires_grad = False
    

In [ ]:
# clap_model.clap.eval()
# clap_model.clap.audio_encoder.eval()
# clap_model.clap.caption_encoder.eval()
torch.cuda.empty_cache()
gc.collect()

0

In [34]:
sample_tensor = torch.randn(16,768)
sample_tensor = sample_tensor.to(0)

audio_embed = clap_model.clap.audio_encoder(sample_tensor)
len(audio_embed)

2

In [ ]:
caption_encoder_config = clap_model.clap.caption_encoder.base.htsat.config
# caption_encoder_config.enable_repeat_mode

In [47]:
audio_encoder_config = clap_model.clap.audio_encoder.base.htsat.config
audio_encoder_config.enable_repeat_mode

False

In [35]:
print(audio_embed[0].shape)
print(audio_embed[1].shape)

torch.Size([16, 1024])
torch.Size([16, 527])


In [ ]:
clap_model.clap.audio_encoder(torch.randn(2,1000))

RuntimeError: Argument #4: Padding size should be less than the corresponding input dimension, but got: padding (512, 512) at dimension 2 of input [2, 1, 440]

In [ ]:
def train_epoch(model, train_loader, optimizer, lr_scheduler, step):
    loss_meter = AvgMeter()
    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for batch in tqdm_object:
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}
        loss = model(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step == "batch":
            lr_scheduler.step()

        count = batch["image"].size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))
    return loss_meter
